In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import json
import jieba
import time
import csv
import random
import sys
import codecs






def main():
        reload(sys)
        sys.setdefaultencoding('utf-8')

headers = {
        'Authority':'www.mafengwo.cn',
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate,br',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Connection': 'keep-alive',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'
        }
#爬取组长数据
def Crawl_leader_of_the_information():
    firstPageUrl = 'http://www.mafengwo.cn/g/11337.html'    #第一页网页地址,小组成员页
    res = requests.get(firstPageUrl, headers=headers)       #get请求网页信息
    soup_bs = BeautifulSoup(res.text, 'lxml')               #BeautifulSoup获取html信息
    dic_leader = {}                #dic_leader用于保存组长信息,不知道组员里是否有组长信息,所以单独爬取
    try:
        leader = soup_bs.select('ul.bd > li:nth-of-type(1) > p:nth-of-type(1) > a')  #新方法,理解成:ul标签,类为bd内的第一个li标签内的第一个p标签内的a组
        dic_leader['name'] = leader[0].text       #获取第一个a组内容,及组长名称
        dic_leader['href'] = 'http://www.mafengwo.cn/wenda' + leader[0].get('href')  #获取a组的href属性
    
    except:
        print('Error')    #请求失败
    return dic_leader
def Crawl_members_of_the_information(i):
    ary_members = []       #储存所有组员的信息
    pageUrl = 'http://www.mafengwo.cn/g/11337.html?action=get_members&page={}&gid=11337'.format(i)   #把i页放入地址,生成json请求地址,可在浏览器的F12内查看
    res = requests.get(pageUrl, headers=headers)     #请求json数据
    try:
        json_html = json.loads(res.text)   #把json数据转换成html,json_html为字典
        html = json_html['html']           #获取json_html字典key:html的数据
        soup_bs = BeautifulSoup(html,'lxml')
        members = soup_bs.find('ul',class_='bd list-member')
        for member in members.find_all('li'):     #爬取组员数据
            dic_member = {}
            member_data = member.select('p > a')
            dic_member['name'] = member_data[0].text   #获取组员名称
            dic_member['href'] = 'http://www.mafengwo.cn/wenda' + member_data[0].get('href')   #获取href后,合并网站头http://www.mafengwo.cn,并加上/wenda,后面请求问答的页面用
            ary_members.append(dic_member)
    except:
        print('Error')   #请求失败
    return ary_members
#请求每个组员提问和回答的数量



def Crawl_question_and_answer(dic_member):
    dic_members =[]
    dic_qmembers=[]
    dic_amembers=[]
    question_url = dic_member['href'][:-5] + '/question.html'  #组合完整的组员提问网页地址
    answer_url = dic_member['href'][:-5] + '/answer.html'      #组合完整的组员回答网页地址
    #question
    res = requests.get(question_url, headers=headers)
    soup_bs = BeautifulSoup(res.text, 'lxml')
    #str_name=soup_bs.find('div',class_='MAvaName').strip('\n').text
    #dic_member['nickname']=str_name
    str_question = soup_bs.find('span',class_='_j_total_item').text
    dic_member['question'] = int(str_question[:-1])        #删除'个'字,并字符转换为整数
    time.sleep(random.randint(2,5))#通过生成随机数,来延时请求,防止反爬,短时间内大量请求会被封IP,导致无法爬取
    str_qsession=soup_bs.find_all('div', attrs={'class':"wd-item"})
    i=1
    for qsession in str_qsession:
        dic_qmember={}
        dic_qmember['Qno.']=i
        dic_member['questionhref']='http://www.mafengwo.cn'+(qsession.find('div',class_='wd-title')).find('a')['href']
        dic_qmember['questiontitle']=qsession.find('a').text
        #dic_qmember['questioncontext']=qsession.find('div',class_='wd-detail').text
        i+=1
        dic_qmembers.append(dic_qmember)
    dic_member['questionhref']=str(dic_qmembers)
    #print(dic_member['Qcontext'])
    #answer
    res = requests.get(answer_url, headers=headers)
    soup_bs = BeautifulSoup(res.text, 'lxml')
    str_answer = soup_bs.find('span',class_='_j_total_item').text
    dic_member['answer'] = int(str_answer[:-1])
    time.sleep(random.randint(2,5))
    str_asession=soup_bs.find_all('div', attrs={'class':"wd-item"})
    q=1
    for asession in str_asession:
        dic_amember={}
        #dic_amember['answer'] = int(str_answer[:-1])
        dic_amember['Ano,']=q
        dic_amember['answerhref']='http://www.mafengwo.cn'+(asession.find('div',class_='wd-title')).find('a')['href']
        dic_amember['answertitle']=asession.find('a').text
        #dic_amember['answercontext']=asession.find('div',class_='wd-detail').text
        q+=1
        dic_amembers.append(dic_amember)
    dic_member['answerhref']=str(dic_amembers)
    dic_members.append(dic_member)
    print(dic_member)


    
#打印无提问和回答数的组员
club_data = []      #全部人员的信息
club_data.append(Crawl_leader_of_the_information())         #爬组长的信息(名字,连接)
for i in range(1,17):
    club_data.extend(Crawl_members_of_the_information(i))#爬组员的信息(名字,连接)

for i in range(0,100):
    Crawl_question_and_answer(club_data[i])                #通过连接,爬取提问和回答数，标题和内容


with open("test.csv", 'w', newline='', encoding='utf-8-sig') as csvfile:
    filednames = ['name','question', 'questionhref', 'answer', 'answerhref','href']
    writer = csv.DictWriter(csvfile, fieldnames=filednames)
    writer.writeheader()
    writer.writerow(club_data)